In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
submission = pd.read_csv('../input/pubg-finish-placement-prediction/sample_submission_V2.csv')
test = pd.read_csv('../input/pubg-finish-placement-prediction/test_V2.csv')
train = pd.read_csv('../input/pubg-finish-placement-prediction/train_V2.csv')

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost
from sklearn.metrics import mean_absolute_error

In [4]:
test.shape,train.shape,submission.shape

In [5]:
df= train.dropna()

In [6]:
df_col= [ 'assists', 'boosts', 'damageDealt', 'DBNOs',
       'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration', 'maxPlace',
       'numGroups', 'rankPoints', 'revives', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints', 'winPlacePerc']

In [7]:
df_train= df.groupby(['groupId'])[df_col].agg('mean').reset_index()

In [8]:
df_col= [ 'assists', 'boosts', 'damageDealt', 'DBNOs',
       'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration', 'maxPlace',
       'numGroups', 'rankPoints', 'revives', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints']

In [9]:
df_test= test.groupby(['groupId'])[df_col].agg('mean').reset_index()

In [10]:
use = test.groupby(['groupId'])[df_col].agg('mean').reset_index()

In [11]:
df_train=df_train.drop(columns=["killPlace","groupId","kills"])

In [12]:
df_test=df_test.drop(columns=['killPlace','groupId','kills'])

In [13]:
use=use.drop(columns=['killPlace','kills'])

In [14]:
check_column=['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals',
        'killStreaks', 'longestKill', 'maxPlace',
       'numGroups', 'revives', 'rideDistance', 'roadKills', 'swimDistance',
       'teamKills', 'vehicleDestroys', 'walkDistance', 'weaponsAcquired']#18

In [15]:
df_t_IQR = df_train.copy()
for i in check_column:
    quantile_25 = np.percentile(df_train[i].values, 25)
    quantile_75 = np.percentile(df_train[i].values, 75)
    
    IQR = quantile_75 - quantile_25
    IQR_weight = IQR*1.5
  
    lowest = quantile_25 - IQR_weight
    highest = quantile_75 + IQR_weight
    outlier_idx = df_t_IQR[i][ (df_t_IQR[i] < lowest) | (df_t_IQR[i] > highest) ].index
    df_t_IQR = df_train.drop(outlier_idx, axis=0)

In [16]:
df_t_IQR2 = use.copy()
for i in check_column:
    quantile_25 = np.percentile(use[i].values, 25)
    quantile_75 = np.percentile(use[i].values, 75)
    
    IQR = quantile_75 - quantile_25
    IQR_weight = IQR*1.5
  
    lowest = quantile_25 - IQR_weight
    highest = quantile_75 + IQR_weight
    outlier_idx = df_t_IQR2[i][ (df_t_IQR2[i] < lowest) | (df_t_IQR2[i] > highest) ].index
    df_t_IQR2 = use.drop(outlier_idx, axis=0)

In [17]:
df_t_IQR1 = df_test.copy()
for i in check_column:
    quantile_25 = np.percentile(df_test[i].values, 25)
    quantile_75 = np.percentile(df_test[i].values, 75)
    
    IQR = quantile_75 - quantile_25
    IQR_weight = IQR*1.5
  
    lowest = quantile_25 - IQR_weight
    highest = quantile_75 + IQR_weight
    outlier_idx = df_t_IQR1[i][ (df_t_IQR1[i] < lowest) | (df_t_IQR1[i] > highest) ].index
    df_t_IQR1 = df_test.drop(outlier_idx, axis=0)

In [18]:
x=df_t_IQR.drop(columns='winPlacePerc')
y=df_t_IQR[['winPlacePerc']]
scaler = StandardScaler()
x=scaler.fit_transform(x)
x_train,x_val,y_train,y_val = train_test_split(x,y,test_size=0.2,random_state=0)

In [19]:
xxx=scaler.fit_transform(df_t_IQR1)
xxx

In [20]:
xgb_model = xgboost.XGBRegressor(
    eval_metric=mean_absolute_error
    ,random_state=0)

xgb_model.fit(x_train,y_train)

In [21]:
Y=xgb_model.predict(xxx)

In [22]:
len(Y)#모델로 타겟값을 뽑아낸것

In [23]:
len(test)#제출용 y값을 붙이는 곳

In [24]:
df_t_IQR2['winPlacePerc']=Y

In [25]:
group =df_t_IQR2.loc[:,['groupId','winPlacePerc']]#작은 테스트 -> 그룹아이디로 test에 붙여야 한다

In [26]:
merge = test.loc[:,'groupId']

In [27]:
merge.shape,group.shape

In [ ]:
group.isnull().sum()

In [33]:
test1=pd.merge(merge,group,how='left',on='groupId')
test1.isnull().sum()
test2=pd.merge(merge,group,how='right',on='groupId')
test2.isnull().sum()

In [34]:
test1.shape,test2.shape

In [29]:
merge

In [30]:
#0.067

In [31]:
# submission에 그 결과값을 담아서 제출하기
submission['winPlacePerc'] = pred_X_test
submission.to_csv('submission.csv', index = False)